# Full EDA pipeline for EEG_Train_Binary.csv

This notebook runs a ready-to-run exploratory data analysis (EDA) pipeline for the EEG CSV dataset (X1..X16 channels + binary target y). It performs basic file inspection, visualization of raw waveforms, univariate summaries, correlation/VIF checks, PCA, PSD/band-power extraction, simple feature engineering, and a baseline model evaluation.

Usage:
- Update CSV_PATH to point to the file on disk (by default it's set to the repository-relative path you provided).
- If you know the sampling frequency (fs) of the EEG recordings, set `FS` accordingly; default is 128 Hz.

Requirements: pandas, numpy, matplotlib, seaborn, scipy, scikit-learn, statsmodels, tqdm

Run all cells from top to bottom.

In [ ]:
# Optional: install missing packages (uncomment if needed)
# !pip install pandas numpy matplotlib seaborn scipy scikit-learn statsmodels tqdm

import os
from pathlib import Path
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.signal import welch
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, accuracy_score
from statsmodels.stats.outliers_influence import variance_inflation_factor
from tqdm.auto import tqdm

sns.set(style='whitegrid')
%matplotlib inline

## Parameters — set these before running


In [ ]:
# Path to CSV. Update if needed.
CSV_PATH = "GU_ML01/Assignment1/EEG_Train_Binary.csv"
# If you prefer pointing to a local copy, change CSV_PATH accordingly, e.g. '/mnt/data/EEG_Train_Binary.csv'

# Sampling frequency (Hz) for PSD / bandpower. Change if you know the actual FS.
FS = 128

# Target column name
TARGET_COL = 'y'

# Output prefix
OUT_PREFIX = Path(CSV_PATH).stem + "_eda"

print('CSV_PATH =', CSV_PATH)
print('FS =', FS)

In [ ]:
# Load CSV
p = Path(CSV_PATH)
if not p.exists():
    # Try to load from current working directory or the raw GitHub URL if file not present
    print(f"File not found at {CSV_PATH}. Please place the CSV in that path or update CSV_PATH.")
else:
    df = pd.read_csv(p)
    print('Loaded:', p, 'shape=', df.shape)

try:
    df
except NameError:
    raise SystemExit('Dataset not found. Set CSV_PATH to the correct file location and re-run.')


## Basic inspection: shape, dtypes, head/tail, missing values, duplicates, class balance, and summary stats


In [ ]:
print('Shape:', df.shape)
print('\nColumns:')
print(df.columns.tolist())
print('\nDtypes:')
print(df.dtypes)

display(df.head())
display(df.tail())

print('\nMissing values per column:')
print(df.isna().sum())
print('\nMissing percent per column:')
print((df.isna().mean()*100).round(3))
print('\nDuplicate rows:', df.duplicated().sum())

if TARGET_COL in df.columns:
    print('\nClass balance for', TARGET_COL)
    print(df[TARGET_COL].value_counts(dropna=False))
    print((df[TARGET_COL].value_counts(normalize=True, dropna=False)*100).round(3))

print('\nSummary statistics (numeric columns):')
display(df.select_dtypes(include=[np.number]).describe().T)


## Visualize raw samples and per-channel summary
- Plot a few example rows (multi-channel waveform per row)
- Plot mean ± std across dataset per channel
- Histograms and boxplots for channels

In [ ]:
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
channels = [c for c in numeric_cols if c != TARGET_COL]
print('Channels detected:', channels)

# Plot first N rows
N = 6
n = min(N, len(df))
fig, axes = plt.subplots(n, 1, figsize=(14, 2.2*n), sharex=True)
for i in range(n):
    axes[i].plot(df.loc[i, channels].values, marker='o')
    axes[i].set_ylabel(f'row {i} (y={df.loc[i, TARGET_COL] if TARGET_COL in df.columns else "NA"})')
    axes[i].grid(True)
axes[-1].set_xticks(range(len(channels)))
axes[-1].set_xticklabels(channels, rotation=45)
plt.suptitle('Example rows: channel values across X1..X16')
plt.tight_layout(rect=[0,0,1,0.97])
plt.show()

# Mean ± std per channel
mean = df[channels].mean()
std = df[channels].std()
plt.figure(figsize=(12,4))
plt.errorbar(range(len(channels)), mean, yerr=std, fmt='-o')
plt.xticks(range(len(channels)), channels, rotation=45)
plt.title('Mean ± std across dataset per channel')
plt.grid(True)
plt.show()

# Histograms
df[channels].hist(bins=40, figsize=(14,10))
plt.suptitle('Channel histograms')
plt.show()

# Boxplot across channels
plt.figure(figsize=(14,6))
sns.boxplot(data=df[channels], orient='h')
plt.title('Boxplots for channels (horizontal)')
plt.show()

## Correlation matrix and VIF
Compute Pearson correlations and Variance Inflation Factor (VIF) to spot multicollinearity.

In [ ]:
corr = df[channels].corr()
plt.figure(figsize=(10,8))
sns.heatmap(corr, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation matrix')
plt.show()

# Top correlated pairs
pairs = []
cols = corr.columns.tolist()
for i in range(len(cols)):
    for j in range(i+1, len(cols)):
        pairs.append((cols[i], cols[j], corr.iloc[i,j]))
pairs_sorted = sorted(pairs, key=lambda x: abs(x[2]), reverse=True)
print('\nTop absolute correlation pairs (top 10):')
for a,b,v in pairs_sorted[:10]:
    print(f"{a} - {b}: {v:.4f}")

# VIF calculation (requires no NaNs)
X = df[channels].fillna(0).values
vif_data = pd.Series([variance_inflation_factor(X, i) for i in range(X.shape[1])], index=channels)
print('\nVIF per channel:')
print(vif_data.sort_values(ascending=False))

## PCA and dimensionality reduction
Standardize, run PCA, plot explained variance and PCA(2) colored by class.

In [ ]:
scaler = StandardScaler()
Xs = scaler.fit_transform(df[channels].fillna(0).values)
pca = PCA()
pca.fit(Xs)
cumvar = np.cumsum(pca.explained_variance_ratio_)
plt.figure(figsize=(8,4))
plt.plot(np.arange(1, len(cumvar)+1), cumvar*100, '-o')
plt.xlabel('n components')
plt.ylabel('Cumulative explained variance (%)')
plt.grid(True)
plt.show()

X_pca2 = PCA(n_components=2).fit_transform(Xs)
plt.figure(figsize=(6,6))
if TARGET_COL in df.columns:
    sns.scatterplot(x=X_pca2[:,0], y=X_pca2[:,1], hue=df[TARGET_COL].astype(str), alpha=0.7)
else:
    plt.scatter(X_pca2[:,0], X_pca2[:,1], alpha=0.7)
plt.title('PCA(2)')
plt.show()

## PSD and bandpower feature extraction
We'll compute Welch PSD for channels and extract bandpower in canonical EEG bands.
Note: FS default is 128 Hz; set FS to the correct sampling rate if known.

In [ ]:
def bandpower(x, sf, band, nperseg=None):
    """Compute band power by integrating the PSD over a frequency band."""
    if nperseg is None:
        nperseg = min(len(x), max(256, int(sf*2)))
    f, Pxx = welch(x, fs=sf, nperseg=nperseg)
    f = np.asarray(f)
    Pxx = np.asarray(Pxx)
    idx = np.logical_and(f >= band[0], f <= band[1])
    if not np.any(idx):
        return 0.0
    return np.trapz(Pxx[idx], f[idx])

bands = {
    'delta': (0.5, 4),
    'theta': (4, 8),
    'alpha': (8, 13),
    'beta': (13, 30),
    'gamma': (30, 45)
}

# Example: plot PSD for first channel
ch0 = channels[0]
f, Pxx = welch(df[ch0].values, fs=FS, nperseg=min(256, len(df[ch0].values)))
plt.semilogy(f, Pxx)
plt.xlim(0, 60)
plt.xlabel('Frequency (Hz)')
plt.ylabel('PSD')
plt.title(f'Welch PSD - {ch0}')
plt.grid(True)
plt.show()

# Compute bandpower per channel for the whole row (i.e., treating each row as a short timeseries of 16 samples ONLY if rows are epochs)
# NOTE: This dataset appears to have each row represent values across 16 channel *samples*, not a long timeseries per channel.
# If each row is an epoch with 16 timepoints per channel, bandpower on 16 samples is unreliable. However, we still compute PSD/bandpower
# for analysis — if your dataset actually places time along columns and channels as rows, you should transpose accordingly.

bandpower_df = pd.DataFrame(index=df.index)
for bname, bband in bands.items():
    vals = []
    for idx in range(len(df)):
        # assemble a pseudo-timeseries by taking values across channels for this row
        # This is dataset-specific: here we compute bandpower across the 16-channel vector
        x = df.loc[idx, channels].values.astype(float)
        try:
            bp = bandpower(x, sf=FS, band=bband, nperseg=min(64, len(x)))
        except Exception:
            bp = np.nan
        vals.append(bp)
    bandpower_df[f'bp_{bname}'] = vals

display(bandpower_df.head())
print('\nBandpower summary:')
display(bandpower_df.describe().T)


## Feature engineering: per-channel simple stats and aggregation
We'll compute mean, std, skew, kurtosis, RMS, energy (sum squares), and zero-crossing count per row across channels, then combine with bandpower features to form a feature table for baseline modelling.

In [ ]:
from scipy.stats import skew, kurtosis

features = pd.DataFrame(index=df.index)

# Per-row aggregate stats across channels
features['ch_mean'] = df[channels].mean(axis=1)
features['ch_std'] = df[channels].std(axis=1)
features['ch_median'] = df[channels].median(axis=1)
features['ch_min'] = df[channels].min(axis=1)
features['ch_max'] = df[channels].max(axis=1)
features['ch_skew'] = df[channels].apply(lambda row: skew(row), axis=1)
features['ch_kurtosis'] = df[channels].apply(lambda row: kurtosis(row), axis=1)
features['ch_rms'] = np.sqrt((df[channels]**2).mean(axis=1))
features['ch_energy'] = (df[channels]**2).sum(axis=1)
features['ch_zc'] = df[channels].apply(lambda row: ((np.diff(np.sign(row.values))!=0).sum()), axis=1)

# Add bandpower features computed earlier
features = pd.concat([features, bandpower_df.add_prefix('row_')], axis=1)

print('Feature matrix shape:', features.shape)
display(features.head())


## Baseline modeling: logistic regression with StratifiedKFold
We will use ch_mean, ch_std, ch_rms, energy, zc and bandpower features as a simple baseline.
All preprocessing (scaling) is done inside CV to avoid leakage.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold

if TARGET_COL not in df.columns:
    print('Target column not found in dataset; skipping baseline modelling.')
else:
    X = features.copy()
    y = df[TARGET_COL].values

    clf = Pipeline([
        ('scaler', StandardScaler()),
        ('lr', LogisticRegression(max_iter=1000))
    ])
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    acc_scores = []
    auc_scores = []
    for train_idx, test_idx in skf.split(X, y):
        Xtr, Xte = X.iloc[train_idx], X.iloc[test_idx]
        ytr, yte = y[train_idx], y[test_idx]
        clf.fit(Xtr, ytr)
        yp = clf.predict(Xte)
        yp_prob = clf.predict_proba(Xte)[:,1] if hasattr(clf.named_steps['lr'], 'predict_proba') else None
        acc_scores.append(accuracy_score(yte, yp))
        if yp_prob is not None:
            auc_scores.append(roc_auc_score(yte, yp_prob))
    print('Baseline LogisticRegression on aggregated features')
    print('Accuracy (5-fold):', np.round(np.mean(acc_scores),4), '±', np.round(np.std(acc_scores),4))
    if auc_scores:
        print('ROC AUC (5-fold):', np.round(np.mean(auc_scores),4), '±', np.round(np.std(auc_scores),4))


## Save features and summary
We save the engineered features and a JSON summary of basic inspection.

In [ ]:
features_out = OUT_PREFIX + '_features.csv'
summary_out = OUT_PREFIX + '_basic_summary.json'

features.to_csv(features_out, index=True)
print('Saved features to', features_out)

basic_summary = {
    'shape': df.shape,
    'columns': df.columns.tolist(),
    'dtypes': df.dtypes.apply(lambda x: str(x)).to_dict(),
    'missing': df.isna().sum().to_dict(),
    'duplicate_rows': int(df.duplicated().sum()),
    'class_balance': df[TARGET_COL].value_counts(dropna=False).to_dict() if TARGET_COL in df.columns else None
}
with open(summary_out, 'w') as f:
    json.dump(basic_summary, f, indent=2)
print('Saved summary to', summary_out)


## Short narrative

This notebook performed a complete EDA pass: file load and basic checks, visual inspection of example rows, distribution and outlier views, correlation and VIF checks, PCA, a (conservative) bandpower computation, per-row aggregation feature engineering, and a baseline logistic regression evaluated with StratifiedKFold. I saved the engineered features and a basic summary JSON. Adjust FS, CSV_PATH, and how bandpower is computed depending on the true data layout (if columns are time samples vs channels).

Next you can:
- If you know the actual sampling frequency and the correct orientation (rows=epochs, columns=timepoints), recompute PSD/bandpower using per-channel time series instead of the current per-row approach.
- Add richer signal features (Hjorth, entropy, wavelet bands) and try tree-based models or CNNs on raw epoch arrays.
- If you have subject/session metadata, implement GroupKFold to avoid leakage.

If you'd like, I can now:
- Convert this notebook into a polished Jupyter Notebook file for direct download, or
- Modify the bandpower extraction to treat columns as time samples per channel if you confirm that's the case.
